In [1]:
from google.colab import files
uploaded = files.upload()

Saving caps_fin.csv to caps_fin.csv


In [2]:
import os
# Menampilkan daftar file di direktori saat ini
print(os.listdir())

['.config', 'caps_fin.csv', 'sample_data']


In [3]:
import pandas as pd

# Muat dataset menggunakan Pandas
data = pd.read_csv('caps_fin.csv')

# Tampilkan keseluruhan dataset
print(data)

                                   Nama Resto  \
0            Soon Fatt Chinese Restaurant JGC   
1                  Sambal Bakar Indonesia JGC   
2             Shu Guo Yin Xiang Kelapa Gading   
3    Layar Seafood Dan Ikan Bakar Sedayu City   
4                        Global Bakery Cakung   
..                                        ...   
534                                 Wing Heng   
535                          Demore Bakehouse   
536                               Jack & John   
537                              Cutt & Grill   
538                     Kopi@Kopi Ground Zero   

                                                alamat         daerah  \
0    Ruko The Boulevard no. 39, Jl. Jkt Garden City...  Jakarta Timur   
1    Ruko Cleon Park Jakarta Garden City, Jl. Jkt G...  Jakarta Timur   
2    Old Shanghai Sedayu City, Unit D2, Cakung Bar....  Jakarta Timur   
3    RT.8/RW.5, Rawa Terate, Cakung, East Jakarta C...  Jakarta Timur   
4    Jl. Tipar Cakung No.2, RT.5/RW.8, Cakung 

In [4]:
# Cek missing values
print(data.isnull().sum())

Nama Resto       0
alamat           0
daerah           0
buka_tutup       0
jam_buka         0
jam_tutup        2
harga_semua      0
harga_awal       0
harga_akhir      1
rata_rata        0
rating           0
telepon         39
tipe             0
Unnamed: 13    539
Unnamed: 14    538
dtype: int64


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
categorical_columns = ['daerah', 'harga_semua', 'tipe']

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[['rating']] = scaler.fit_transform(data[['rating']])

In [8]:
from sklearn.model_selection import train_test_split

# Pilih fitur yang akan digunakan untuk rekomendasi
features = ['daerah', 'harga_semua', 'rating', 'tipe']

# Split data into train and remaining (validation + test)
train_data, remaining_data = train_test_split(data, test_size=0.3, random_state=42)

# Split remaining data into validation and test
validation_data, test_data = train_test_split(remaining_data, test_size=0.5, random_state=42)

# Print shapes to verify
print(f"Train Data: {train_data.shape}")
print(f"Validation Data: {validation_data.shape}")
print(f"Test Data: {test_data.shape}")

Train Data: (377, 15)
Validation Data: (81, 15)
Test Data: (81, 15)


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Input dimension based on features
input_dim = len(features)

# Input layer
input_layer = Input(shape=(input_dim,))

# Encoder
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

# Latent representation
latent = Dense(16, activation='relu')(encoded)

# Decoder
decoded = Dense(32, activation='relu')(latent)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

# Model
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

# Ringkasan model
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 128)               640       
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 32)                544       
                                                                 
 dense_5 (Dense)             (None, 64)                2112  

In [10]:
# Extract feature values
train_features = train_data[features].values
validation_features = validation_data[features].values
test_features = test_data[features].values

# Train the model
history = autoencoder.fit(train_features, train_features,
                          epochs=50,
                          batch_size=32,
                          validation_data=(validation_features, validation_features))

Epoch 1/50
12/12 [==============================] - 2s 26ms/step - loss: 101.3347 - val_loss: 86.3594
Epoch 2/50
12/12 [==============================] - 0s 7ms/step - loss: 99.6629 - val_loss: 86.0118
Epoch 3/50
12/12 [==============================] - 0s 8ms/step - loss: 99.5094 - val_loss: 85.9689
Epoch 4/50
12/12 [==============================] - 0s 8ms/step - loss: 99.4876 - val_loss: 85.9653
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 99.4849 - val_loss: 85.9647
Epoch 6/50
12/12 [==============================] - 0s 7ms/step - loss: 99.4843 - val_loss: 85.9654
Epoch 7/50
12/12 [==============================] - 0s 6ms/step - loss: 99.4841 - val_loss: 85.9645
Epoch 8/50
12/12 [==============================] - 0s 8ms/step - loss: 99.4839 - val_loss: 85.9647
Epoch 9/50
12/12 [==============================] - 0s 7ms/step - loss: 99.4837 - val_loss: 85.9641
Epoch 10/50
12/12 [==============================] - 0s 8ms/step - loss: 99.4835 - val_loss: 85.96

In [11]:
# Model untuk mendapatkan representasi latent
encoder = Model(input_layer, latent)

# Mendapatkan representasi latent dari data restoran
restaurant_latent = encoder.predict(data[features].values)


17/17 [==============================] - 0s 2ms/step


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Hitung kemiripan antar restoran
similarity_matrix = cosine_similarity(restaurant_latent)


In [14]:
def get_similar_restaurants(restaurant_name, data, similarity_matrix, top_n=5):
    # Cari indeks restoran berdasarkan nama
    idx = data[data['Nama Resto'] == restaurant_name].index[0]

    # Ambil skor kemiripan restoran tersebut dengan restoran lainnya
    similarity_scores = list(enumerate(similarity_matrix[idx]))

    # Urutkan berdasarkan skor kemiripan (skor tertinggi di atas)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Ambil top_n restoran yang paling mirip (selain restoran itu sendiri)
    similarity_scores = similarity_scores[1:top_n+1]

    # Ambil indeks restoran yang paling mirip
    similar_restaurant_indices = [i[0] for i in similarity_scores]

    # Kembalikan data restoran yang mirip
    return data.iloc[similar_restaurant_indices]

# Contoh penggunaan
similar_restaurants = get_similar_restaurants('BRAUD', data, similarity_matrix)
print(similar_restaurants)


                                   Nama Resto  \
210                        Paul Pacific Place   
212         Ann's Bakehouse & Creamery Sambas   
226                        Sophie Authentique   
76   Majestyk Bakery & Cake Shop Pondok Bambu   
77                Holland Bakery Pondok Bambu   

                                                alamat  daerah  \
210  Pacific Place Mall, Lantai Ground, South Lobby...       2   
212  Jl. Sungai Sambas 3 No.12, RT.8/RW.5, Kramat P...       2   
226  Jl. Kemang Raya No.55 4, RT.4/RW.2, Bangka, Ke...       2   
76   Jl. Pahlawan Revolusi No.3, RT.1/RW.7, Pd. Bam...       3   
77   Jl. Pahlawan Revolusi No.10 2, RT.2/RW.5, Pd. ...       3   

          buka_tutup     jam_buka jam_tutup  harga_semua  harga_awal  \
210  06.30  -  22.00  06:30:00 AM     22.00            5      100000   
212  08.00  -  21.00  08:00:00 AM     21.00            5      100000   
226  08.00  -  17.00  08:00:00 AM     17.00            5      100000   
76   07.00  -  21.00

In [15]:
# Decode categorical columns
for col in ['harga_semua', 'tipe', 'daerah']:
    le = label_encoders[col]
    try:
        similar_restaurants[col] = le.inverse_transform(similar_restaurants[col])
    except ValueError as e:
        print(f"Error: {e}")
        print(f"Handling unseen labels for column {col}")
        # Handle unseen labels
        unseen_mask = ~similar_restaurants[col].isin(le.classes_)
        similar_restaurants.loc[unseen_mask, col] = 'Unknown'

# Print recommendations
print(similar_restaurants)

                                   Nama Resto  \
210                        Paul Pacific Place   
212         Ann's Bakehouse & Creamery Sambas   
226                        Sophie Authentique   
76   Majestyk Bakery & Cake Shop Pondok Bambu   
77                Holland Bakery Pondok Bambu   

                                                alamat           daerah  \
210  Pacific Place Mall, Lantai Ground, South Lobby...  Jakarta Selatan   
212  Jl. Sungai Sambas 3 No.12, RT.8/RW.5, Kramat P...  Jakarta Selatan   
226  Jl. Kemang Raya No.55 4, RT.4/RW.2, Bangka, Ke...  Jakarta Selatan   
76   Jl. Pahlawan Revolusi No.3, RT.1/RW.7, Pd. Bam...    Jakarta Timur   
77   Jl. Pahlawan Revolusi No.10 2, RT.2/RW.5, Pd. ...    Jakarta Timur   

          buka_tutup     jam_buka jam_tutup          harga_semua  harga_awal  \
210  06.30  -  22.00  06:30:00 AM     22.00  100.000  -  200.000      100000   
212  08.00  -  21.00  08:00:00 AM     21.00  100.000  -  200.000      100000   
226  08.00  - 

<ipython-input-15-29b5000b352c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_restaurants[col] = le.inverse_transform(similar_restaurants[col])
<ipython-input-15-29b5000b352c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_restaurants[col] = le.inverse_transform(similar_restaurants[col])
<ipython-input-15-29b5000b352c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in